## Calculating diatom limitation 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from glob import glob
from subprocess import check_output

import yaml
import dask

import util

USER = os.environ['USER']

assert os.path.exists('/glade/campaign'), (
    'campaign is not accessible; run on Casper'
)

In [4]:
import warnings
warnings.simplefilter("ignore") # Silence warnings
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import numpy as np
from matplotlib.pyplot import figure

import numpy.ma as ma
from netCDF4 import Dataset as NetCDFFile
import glob
import cartopy
import cartopy.crs as ccrs
import pylab 
from copy import deepcopy
import pandas as pd
import matplotlib
import matplotlib.colors as colors
from cartopy.util import add_cyclic_point
plt.rcParams['mathtext.default']='regular'
from collections import OrderedDict
import cmocean
import matplotlib.cm as cm
import matplotlib as mpl
import seaborn as sns
from matplotlib.gridspec import GridSpec
import matplotlib
# Scientific libraries
from numpy import arange,array,ones
from scipy import stats
import os

In [5]:
import re
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [6]:
cluster, client = util.get_ClusterClient(walltime='24:00:00')
cluster.scale(32)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/gabyn/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/gabyn/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.49:45784,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/gabyn/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


### Import interannual dataset [All 63 years of the 2$^{nd}$ cycle]
____ 

In [22]:
path = '/glade/campaign/cesm/development/bgcwg/projects/marbl-spectra/g.e21.G1850ECOIAF.t62_g17.marbl0_33.GNG595/ocn/hist'
case = 'g.e21.G1850ECOIAF.t62_g17.marbl0_33.GNG595.pop.h.'

In [23]:
variables = ['diat2_P_lim_Cweight_avg_100m',
             'diat2_Fe_lim_Cweight_avg_100m',
             'diat2_N_lim_Cweight_avg_100m', 
             'diat2_light_lim_Cweight_avg_100m',
             'TEMP',
             'diat2_SiO3_lim_Cweight_avg_100m',
             'diat2_light_lim_surf']
coords = {'x':'TLONG','y':'TLAT'}
keep_vars = variables + list(coords.values())+['dz','KMT','time']

In [24]:
yr = ['062','063','064','065','066','067','068','069',
     '070','071','072','073','074','075','076','077',
     '078','079','080','081','082','083','084','085',
     '086','087','088','089','090','091','092','093',
     '094','095','096','097','098','099','100','101',
     '102','103','104','105','106','107','108','109',
     '110','111','112','113','114','115','116','117',
     '118','119','120','121','122','123','124']

In [25]:
%%time

ds_ann = xr.Dataset()

for year in np.arange(0,63,1):
    yr4=yr[year]
    print(yr4)
    
    ds = xr.Dataset()
    file = sorted(glob.glob(f'{path}/{case}0{yr4}*.nc'))
    dsv=xr.open_mfdataset(file, decode_times=True,drop_variables=["transport_components", "transport_regions"], 
                            parallel=True, compat="override", combine='nested', concat_dim="time",data_vars="minimal",coords='minimal' )
    for vv in variables: 
        #print('got dsv')
        ds = xr.merge((ds, dsv[vv]))   

    ds = ds.drop([v for v in ds.variables if v not in keep_vars]).squeeze()
    #print('monthly file', len(ds.time))
    #ds = ds.mean(dim='time')
    ds_ann = xr.concat([ds_ann, ds], dim="year",data_vars="different",join="override")

062
063
064
065
066
067
068
069
070
071
072
073
074
075
076
077
078
079
080
081
082
083
084
085
086
087
088
089
090
091
092
093
094
095
096
097
098
099
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
CPU times: user 5min 53s, sys: 21.3 s, total: 6min 14s
Wall time: 9min 49s


#### small diatom nutrient limitation 

In [26]:
%%time
diat_lims=np.stack([ds_ann.diat2_P_lim_Cweight_avg_100m,
                   ds_ann.diat2_Fe_lim_Cweight_avg_100m,
                   ds_ann.diat2_N_lim_Cweight_avg_100m,
                   ds_ann.diat2_SiO3_lim_Cweight_avg_100m],axis=4)

diat_lims_value=np.zeros(shape=ds_ann.diat2_light_lim_Cweight_avg_100m.shape)
diat_lims_index=np.zeros(shape=ds_ann.diat2_light_lim_Cweight_avg_100m.shape)

for i,t in enumerate(ds_ann.year.values):
        temp=diat_lims[i,:,:,:]
        diat_lims_value[i,:,:]=deepcopy(np.amin(temp,axis=3))
        diat_lims_index[i,:,:]=deepcopy(np.argmin(temp,axis=3))
        
ds_ann['diat2_lims_value']=xr.DataArray(data=diat_lims_value, coords={'year':ds_ann.year.values}, dims=['year','time','nlat','nlon'], 
                                 attrs={'long_name':'diatom 2 maximum limitation value, carbon biomass weighted average over 0-100m'})
ds_ann['diat2_lims_index']=xr.DataArray(data=diat_lims_index, coords={'year':ds_ann.year.values}, dims=['year','time','nlat','nlon'], 
                                 attrs={'long_name':'diatom 2 maximum limitation type, carbon biomass weighted average over 0-100m',
                                        'values':'0-light,1-temp, 2-P, 3-Fe, 4-N, 5-Si'})

CPU times: user 25.4 s, sys: 6.31 s, total: 31.7 s
Wall time: 52.7 s


In [27]:
ds_ann

,Array,Chunk
Bytes,354.38 MiB,0.94 MiB
Shape,"(63, 12, 384, 320)","(2, 1, 384, 320)"
Count,1649 Graph Layers,744 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,354.38 MiB,0.94 MiB
Shape,"(63, 12, 384, 320)","(2, 1, 384, 320)"
Count,1649 Graph Layers,744 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


### Now find the most limiting factor using the light and temperature values
____

### Calcualte the temperature limiation term! 
$$r = r_o e^{\frac{-E_a(T_o -T)}{k T_o T}}$$
$$\frac{\mu}{\mu_o} = e^{\frac{-E_a(T_o -T)}{k T_o T}}$$ 

to simplify however, we will create $\alpha = \frac{E_a}{k}$ and $\gamma_T = 
\frac{\mu}{\mu_o}$ so that: 
$$\gamma_T = e^{ \frac{-\alpha(T_o - T)}{T_o T}}$$

$T_o = 298.15 ^oK$
* it is close to the temperature at which many physiological experiments are run, and it is the mean temperature of the subtropical gyres, which represent a large proportion of the world’s open oceans.

$k = 8.617e^{-5} eVK^{-1}$ 

$E_a = 0.317$ phytoplankton (Kremer et al., 2017).

$E_a = 0.42$ picoplankton 
* Multiple studies have shown that picoplankton have a higher temperature sensitivity compared to phytoplankton of larger sizes (Chen et al., 2014; Stawiarski et al., 2016).

$E_a = 0.65$ zooplankton 
* MTE predicts that heterotrophic organisms will have a greater temperature dependence than autotrophs, as Rubisco carboxylation (rate limiting for photosynthesis) has a lower activation energy (Ea) than ATP synthesis (Allen et al., 2005; Lopez-Urrutia et al., 2006).

and we will have to convert all the surface temperatures from $^oC$ to $K$ and the temperature on the equation and then solve for every gorup... 
$$0°C + 273.15 = 273.15K$$

In [28]:
# convert celcius to kelvin 
temp_K = ds_ann.TEMP + 273.15

# constants 
𝑘=8.617e-5
𝐸𝑎_phyto = 0.317 
𝐸𝑎_pico  = 0.42
𝐸𝑎_zoo   = 0.65
𝑇𝑜 = 298.15

# create simpler variables. 
𝛼_phyto = 𝐸𝑎_phyto/𝑘
𝛼_pico = 𝐸𝑎_pico/𝑘

# calculate temperature limitation term
𝛾𝑇_phyto = np.exp((-𝛼_phyto*(𝑇𝑜-temp_K))/(𝑇𝑜*temp_K))
𝛾𝑇_pico = np.exp((-𝛼_pico*(𝑇𝑜-temp_K))/(𝑇𝑜*temp_K))

In [29]:
ds_ann

,Array,Chunk
Bytes,354.38 MiB,0.94 MiB
Shape,"(63, 12, 384, 320)","(2, 1, 384, 320)"
Count,1649 Graph Layers,744 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,354.38 MiB,0.94 MiB
Shape,"(63, 12, 384, 320)","(2, 1, 384, 320)"
Count,1649 Graph Layers,744 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [30]:
diat2_lim = np.stack([ds_ann.diat2_light_lim_surf, 
                       𝛾𝑇_phyto[:,:,0,:,:], ds_ann.diat2_lims_value],axis=4)

#### small diatom growth limitation 

In [31]:
%%time
diat2_lim_value=np.zeros(shape=ds_ann.diat2_light_lim_Cweight_avg_100m.shape)
diat2_lim_index=np.zeros(shape=ds_ann.diat2_light_lim_Cweight_avg_100m.shape)

for i,t in enumerate(ds_ann.year.values):
        temp=diat2_lim[i,:,:,:,:]
        diat2_lim_value[i,:,:,:]=deepcopy(np.amin(temp,axis=3))
        diat2_lim_index[i,:,:,:]=deepcopy(np.argmin(temp,axis=3))
        
ds_ann['diat2_lim_value']=xr.DataArray(data=diat2_lim_value, coords={'year':ds_ann.year.values}, dims=['year','time','nlat','nlon'], 
                                 attrs={'long_name':'diatom 2 maximum limitation value, carbon biomass weighted average over 0-100m'})
ds_ann['diat2_lim_index']=xr.DataArray(data=diat2_lim_index, coords={'year':ds_ann.year.values}, dims=['year','time','nlat','nlon'], 
                                 attrs={'long_name':'diatom 2 maximum limitation type, carbon biomass weighted average over 0-100m',
                                        'values':'0-light,1-temp, 2-Nutrient'})

CPU times: user 8.93 s, sys: 1.11 s, total: 10 s
Wall time: 10.5 s


In [32]:
diat_lim = xr.Dataset()
diat_lim['diat2_lim_value'] = ds_ann.diat2_lim_value.rename({'year': 'time','time': 'month','nlat': 'nlat','nlon': 'nlon'})
diat_lim['diat2_lim_index'] = ds_ann.diat2_lim_index.rename({'year': 'time','time': 'month','nlat': 'nlat','nlon': 'nlon'})

In [33]:
new_filename_1 = '/glade/u/home/gabyn/scratch/SPECTRA/growth_lim/diat2_limitation.nc'
diat_lim.to_netcdf(path=new_filename_1)

## Now that I have saved it into a new netCDF, I will regrid this nc file to be able to use it for this figure. I will do this by: 
    - module load nco 
    - module load cdo 

`ncatted -a coordinates,diat3_lim_index,c,c,"TLONG TLAT" diat_lim.nc`

`ncatted -a coordinates,diat3_lim_value,c,c,"TLONG TLAT" diat_lim.nc`

`cdo remapbil,r360x180 -selname,diat3_lim_index diat_lim.nc diat3_lim_index_remap.nc`

`cdo remapbil,r360x180 -selname,diat3_lim_value diat_lim.nc diat3_lim_value_remap.nc`

`cdo sellonlatbox,-180,180,-89.5,89.5 diat3_lim_index_remap.nc regrid_diat3_lim_index.nc`

`cdo sellonlatbox,-180,180,-89.5,89.5 diat3_lim_value_remap.nc regrid_diat3_lim_value.nc`

In [34]:
client.close()
cluster.close()